# 7.1 - MNIST Classification using CNN with TensorBoard

## 순서
1. 라이브러리 임포트 및 TensorBoard 설정
2. MNIST 데이터 불러오기 및 전처리
3. CNN 모델 정의 (tf.keras.Model)
4. 손실함수, 옵티마이저, 정확도 함수 정의
5. TensorBoard 로그 생성
6. train_step 함수에서 텐서보드 로그 기록
7. 학습 루프 (Mini-batch)
8. 최종 테스트 정확도 기록 및 TensorBoard에서 시각화


In [1]:
import tensorflow as tf
import datetime, os

%load_ext tensorboard

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.10.1


## 2. MNIST 데이터 불러오기 및 전처리

In [2]:
# MNIST 데이터를 다운로드 하고, 정규화
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

x_train = x_train.reshape([-1, 784])
x_test  = x_test.reshape([-1, 784])

x_train = x_train / 255.
x_test  = x_test / 255.

y_train = tf.one_hot(y_train, depth=10)
y_test  = tf.one_hot(y_test,  depth=10)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

x_train shape: (60000, 784)
y_train shape: (60000, 10)


## 3. CNN 모델 정의 (tf.keras.Model 사용)

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫 번째 Convolution Layer: 5x5 Kernel, 32개 Filter
        self.conv_layer_1 = tf.keras.layers.Conv2D(
            filters=32, kernel_size=5, strides=1, padding='same', activation='relu')
        self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

        # 두 번째 Convolution Layer: 5x5 Kernel, 64개 Filter
        self.conv_layer_2 = tf.keras.layers.Conv2D(
            filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
        self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)

        # Flatten + Fully connected
        self.flatten_layer = tf.keras.layers.Flatten()
        self.fc_layer_1    = tf.keras.layers.Dense(1024, activation='relu')
        self.output_layer  = tf.keras.layers.Dense(10, activation=None)

    def call(self, x):
        # x: (batch, 784) -> reshape to (batch,28,28,1)
        x_image = tf.reshape(x, [-1, 28, 28, 1])

        h_conv1 = self.conv_layer_1(x_image)   # -> (batch, 28,28,32)
        h_pool1 = self.pool_layer_1(h_conv1)   # -> (batch, 14,14,32)

        h_conv2 = self.conv_layer_2(h_pool1)   # -> (batch, 14,14,64)
        h_pool2 = self.pool_layer_2(h_conv2)   # -> (batch, 7,7,64)

        h_pool2_flat = self.flatten_layer(h_pool2)  # -> (batch, 7*7*64=3136)
        h_fc1 = self.fc_layer_1(h_pool2_flat)       # -> (batch, 1024)

        logits = self.output_layer(h_fc1)           # -> (batch, 10)
        y_pred = tf.nn.softmax(logits, axis=1)
        return y_pred, logits

# 모델 인스턴스 생성
CNN_model = CNN()

## 4. 손실함수, 옵티마이저, 정확도 함수 정의

In [4]:
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=y)
    )

optimizer = tf.optimizers.Adam(1e-4)

@tf.function
def compute_accuracy(y_pred, y):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

## 5. TensorBoard 로그 디렉토리 설정

In [5]:
log_dir = "tensorboard_log"  # 로그 저장 경로
train_log_dir = os.path.join(log_dir, "train")
test_log_dir  = os.path.join(log_dir, "test")

train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer  = tf.summary.create_file_writer(test_log_dir)

print(f"train_log_dir: {train_log_dir}")
print(f"test_log_dir : {test_log_dir}")

train_log_dir: tensorboard_log\train
test_log_dir : tensorboard_log\test


## 6. train_step 함수에서 텐서보드 로그 기록

In [6]:
@tf.function
def train_step(model, x, y, step):
    with tf.GradientTape() as tape:
        y_pred, logits = model(x)
        loss_value = cross_entropy_loss(logits, y)

    gradients = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 텐서보드에 loss 기록
    with train_summary_writer.as_default():
        tf.summary.scalar('training_loss', loss_value, step=step)

        x_image = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('training_images', x_image, step=step, max_outputs=3)

    return loss_value

## 7. 학습 루프 (Mini-batch)
- 총 10000 스텝 학습
- 100 스텝마다 현재 정확도, 손실 출력


In [7]:
num_steps = 2000

for i in range(num_steps):
    batch_x, batch_y = next(train_data_iter)

    if i % 100 == 0:
        y_pred_temp, logits_temp = CNN_model(batch_x)
        train_loss = cross_entropy_loss(logits_temp, batch_y)
        train_acc  = compute_accuracy(y_pred_temp, batch_y)
        print(f"[Step {i}] Training Accuracy: {train_acc.numpy():.4f}, Loss: {train_loss.numpy():.4f}")

    # 학습 진행
    loss_val = train_step(CNN_model, batch_x, batch_y, step=i)

[Step 0] Training Accuracy: 0.1400, Loss: 2.3115
Instructions for updating:
Use fn_output_signature instead
[Step 100] Training Accuracy: 0.9000, Loss: 0.4529
[Step 200] Training Accuracy: 0.9000, Loss: 0.2788
[Step 300] Training Accuracy: 0.8800, Loss: 0.3283
[Step 400] Training Accuracy: 0.9200, Loss: 0.1986
[Step 500] Training Accuracy: 0.9000, Loss: 0.3468
[Step 600] Training Accuracy: 0.9400, Loss: 0.1285
[Step 700] Training Accuracy: 0.9400, Loss: 0.1067
[Step 800] Training Accuracy: 0.9800, Loss: 0.1255
[Step 900] Training Accuracy: 0.9800, Loss: 0.1365
[Step 1000] Training Accuracy: 0.9800, Loss: 0.0491
[Step 1100] Training Accuracy: 1.0000, Loss: 0.0288
[Step 1200] Training Accuracy: 0.9600, Loss: 0.1173
[Step 1300] Training Accuracy: 1.0000, Loss: 0.0326
[Step 1400] Training Accuracy: 0.9600, Loss: 0.1034
[Step 1500] Training Accuracy: 1.0000, Loss: 0.0139
[Step 1600] Training Accuracy: 0.9800, Loss: 0.1095
[Step 1700] Training Accuracy: 1.0000, Loss: 0.0211
[Step 1800] Train

## 8. 학습 완료 후 테스트 정확도 측정 및 TensorBoard에 기록

In [8]:
# 최종 테스트 정확도 계산
y_pred_test, logits_test = CNN_model(x_test)
test_acc = compute_accuracy(y_pred_test, y_test)
print(f"최종 테스트 정확도: {test_acc.numpy():.4f}")

# 텐서보드에 테스트 정확도 기록
final_step = num_steps
with test_summary_writer.as_default():
    tf.summary.scalar('test_accuracy', test_acc, step=final_step)

최종 테스트 정확도: 0.9853


In [9]:
%tensorboard --logdir ./tensorboard_log

Reusing TensorBoard on port 6006 (pid 34472), started 0:02:21 ago. (Use '!kill 34472' to kill it.)